![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [46]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# Import any additional modules and start coding below
rental_info = pd.read_csv("rental_info.csv")
#print(rental_info.head())
rental_info["rental_length"] = pd.to_datetime(rental_info["return_date"]) - pd.to_datetime(rental_info["rental_date"])
rental_info["rental_length_days"] = rental_info["rental_length"].dt.days
#print(rental_info["rental_length_days"])
rental_info["deleted_scenes"] = np.where(rental_info["special_features"].str.contains("Deleted Scenes"),1,0)
rental_info["behind_the_scenes"] = np.where(rental_info["special_features"].str.contains("Behind the Scenes"),1,0)
#print(rental_info[["Deleted_Scenes","behind_the_scenes"]])
cols_to_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]
X = rental_info.drop(cols_to_drop, axis = 1)
y = rental_info["rental_length_days"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 9)
lasso = Lasso(alpha = 0.1, random_state=9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
X_lasso_train, X_lasso_test = X_train.iloc[:,lasso_coef > 0], X_test.iloc[:,lasso_coef > 0]
ols = LinearRegression()
ols.fit(X_lasso_train, y_train)
y_pred = ols.predict(X_lasso_test)
ols_MSE = np.sqrt(MSE(y_test, y_pred))
print("Mean Squared error of linear regression", ols_MSE)
rf = RandomForestRegressor()
param_rf= {'n_estimators': np.arange(1,101,1),
          'max_depth' : np.arange(1,11,1)}
grid_rf = RandomizedSearchCV(rf, 
                                 param_distributions=param_rf, 
                                 cv=5, 
                                 random_state=9
)
grid_rf.fit(X_train, y_train)
grid_rf_best_model = grid_rf.best_estimator_
grid_rf_best_model.fit(X_train, y_train)
grid_rf_best_model_pred = grid_rf_best_model.predict(X_test)
grid_rf_best_model_MSE = np.sqrt(MSE(y_test, grid_rf_best_model_pred))
print("Mean Squared error of Randomized CV Search", grid_rf_best_model_MSE)
hyperparameter_rf = grid_rf.best_params_
rf = RandomForestRegressor(n_estimators = hyperparameter_rf['n_estimators'],
                          max_depth= hyperparameter_rf['max_depth'],
                          random_state= 9)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_MSE = np.sqrt(MSE(y_test, rf_pred))
print("Mean Squared error of Random Forest", grid_rf_best_model_MSE)

best_model = rf
best_mse = rf_MSE

Mean Squared error of linear regression 2.1936948833591794
Mean Squared error of Randomized CV Search 1.4887026900397242
Mean Squared error of Random Forest 1.4887026900397242
